## CCFRP data conversion

In [16]:
## Imports

import pandas as pd
import numpy as np
import random

from datetime import datetime # for handline dates
import pytz # for handling time zones

import urllib.request, urllib.parse, json # for dealing with WoRMS API and output

In [3]:
## Load csv

path = 'C:\\Users\\dianalg\\Documents\\Work\\MBARI\\MPA Data Integration\\CCFRP\\'
filename = 'Counts.csv'
data = pd.read_csv(path+filename)

data.head()

,Area,Site,Lat Center Point,Lon Center Point,Year,Barred Sand Bass,Bat Ray,Bigmouth Sole,Black-and-Yellow Rockfish,Black Rockfish,...,Vermilion Rockfish,White Croaker,White Seabass,Widow Rockfish,Wolf Eel,Yelloweye Rockfish,Yellowfin Croaker,Yellowtail Jack,Yellowtail Rockfish,Total
0,Trinidad,REF,41.115,-124.173,2018,0,0,0,0,708,...,2,0,0,0,0,0,0,0,22,898
1,Trinidad,REF,41.115,-124.173,2019,0,0,0,0,384,...,1,0,0,0,0,1,0,0,16,504
2,Cape Mendocino,MPA,40.426,-124.478,2017,0,0,0,0,113,...,9,0,0,0,0,3,0,0,4,229
3,Cape Mendocino,MPA,40.426,-124.478,2018,0,0,0,0,58,...,20,0,0,0,0,10,0,0,7,300
4,Cape Mendocino,MPA,40.426,-124.478,2019,0,0,0,0,52,...,15,0,0,0,0,6,0,0,10,234


Patrick suggested that I start by mapping all species names to a WoRMS ID.

In [23]:
## Get species names

common_names = data.columns[5:-1].to_series()
common_names.reset_index(drop=True, inplace=True)
common_names

0              Barred Sand Bass
1                       Bat Ray
2                 Bigmouth Sole
3     Black-and-Yellow Rockfish
4                Black Rockfish
                ...            
85                     Wolf Eel
86           Yelloweye Rockfish
87            Yellowfin Croaker
88              Yellowtail Jack
89          Yellowtail Rockfish
Length: 90, dtype: object

In [41]:
## Try on a subset of the names first:

# Seed random number generator for consistent results
random.seed(42)

# Get a set of 10 random indices
idx = [random.randint(0, len(common_names)) for i in range(10)]

# Get subset
subset = common_names[idx]
subset

81           Vermilion Rockfish
14        California Lizardfish
3     Black-and-Yellow Rockfish
35                Kelp Rockfish
31                Jack Mackerel
28               Grass Rockfish
17              Canary Rockfish
13           California Halibut
86           Yelloweye Rockfish
69             Speckled Sanddab
dtype: object

Create helper function to get scientic name, scientific name ID, taxon ID from common name using WoRMS. This function is based on the one written by Luke at Axiom (possibly edited by Patrick) in ACCESS_Trawls_to_dwc.ipynb.

**STUCK ON THIS STEP**

In [ ]:
def worms_from_common_name(common_name):
    """
    Using the WoRMS REST API, retrieve WoRMS ID, scientific name and taxon ID given a common name.
    
    Dependencies:
        import urllib.request, urllib.parse, json
    
    Usage:
        worms_from_common_name(common_name)
        
    Inputs:
        The common name of interest as a string.
        
    Outputs:
        1. scientificName: WoRMS specified scientific name
        2. scientificNameID: WoRMS specified ID
        3. taxonID: WoRMS specified taxon ID
    """
    
    name_url = urllib.parse.quote(common_name)
    _url = 'http://www.marinespecies.org/rest/AphiaRecordsByNames?scientificnames%5B%5D='+ name_url + '&like=false&marine_only=false'
    
    try:
        with urllib.request.urlopen(_url) as url:
            data = json.loads(url.read().decode())
            return (data[0][0]['scientificname'], data[0][0]['lsid'], data[0][0]['AphiaID'])
    except Exception as e:
        # Try passing just the genus if there the species name is unrecognized
        if len(sci_name_url.split('%20')) > 1: #If species is unknown and listed as spp. or sp.
            return get_worms_from_scientific_name(sci_name_url.split('%20')[0])
        else:
            print("Url didn't work, check name, ", sci_name)

In [42]:
test = subset.iloc[3]
test

'Kelp Rockfish'

In [43]:
test_url = urllib.parse.quote(test)

In [45]:
 _url = 'http://www.marinespecies.org/rest/AphiaRecordsByNames?commonnames%5B%5D='+ test_url + '&like=false&marine_only=false'
_url

'http://www.marinespecies.org/rest/AphiaRecordsByNames?commonnames%5B%5D=Kelp%20Rockfish&like=false&marine_only=false'

In [34]:
with urllib.request.urlopen(_url) as url:
            data = json.loads(url.read().decode())
            print(data[0][0]['scientificname'], data[0][0]['lsid'], data[0][0]['AphiaID'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)